In [1]:
import numpy as np
import pandas as pd
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import time
import datetime as dt
from time import sleep
from IPython.display import clear_output
import os
from selenium import webdriver
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver import Keys, ActionChains
from selenium.webdriver.common.actions.action_builder import ActionBuilder

In [25]:
def txt_to_links(text_file):
    with open(text_file, 'r', encoding="utf-8") as f:
        lista_aptos_raw = f.read()
    soup = BeautifulSoup(lista_aptos_raw, 'html.parser')
    data_raw = soup.find(attrs={"class": "Ul-sctud2-0 jyGHXP realestate-results-list browse-results-list"})
    data_raw = data_raw.find_all(attrs={"class": "sc-bdVaJa ebNrSm"})
    list_links = list()
    for data in data_raw:
        try:
            list_links.append('https://www.metrocuadrado.com'+data['href'])
        except: None
    out = list(set(list_links))

    print(f'{text_file}. Enlaces importados: {len(out)}')
    return out

def enlace_to_rowdf(enlace):
    new_row = dict()
    new_row['Código inmueble'] = 'UNASIG 000'
    new_row['Enlace'] = enlace
    obs_text = ''
    try:
        page = urlopen(enlace)
        html_bytes = page.read()
        html = html_bytes.decode("utf-8")
        soup = BeautifulSoup(html, 'html.parser')

        try:
            data_raw_1 = soup.find('title')
            new_row['Tipo'] = data_raw_1.text.split(' ')[2]
        except:
            obs_text = obs_text + 'sin info_2'

        try:
            data_raw_2 = soup.find('p', attrs={"class": "P-sc-31hrrf-0 hGwghD card-subtitle"})
            new_row['Sector'] = ', '.join(data_raw_2.text.split(',')[0].upper().replace('SECTOR ', '').replace(' Y ALREDEDORES', '').split(' Y '))
        except:
            obs_text = obs_text + 'sin info_2'

        try:
            data_raw_3 = soup.find('ul', attrs={"class": "Ul-sctud2-0 kdIYsW inline-list-grid list-feature-detail"})
            data_raw_3 = data_raw_3.find_all('li')
            new_row['Habitaciones'] = int(data_raw_3[1].text[0])
            new_row['Baños'] = int(data_raw_3[2].text[0])
            new_row['Estrato'] = int(data_raw_3[3].text[0])
        except:
            new_row['Habitaciones'] = np.NaN
            new_row['Baños'] = np.NaN
            new_row['Estrato'] = np.NaN
            obs_text = obs_text + ', '+ 'sin info_3'

        try:
            data_raw_4 = soup.find_all(attrs={"class": "Col-sc-14ninbu-0 lfGZKA mb-3 pb-1 col-12 col-lg-3"})
            for element in data_raw_4:
                if element.h3.text == 'Precio' or element.h3.text == 'Valor administración':
                    text = int(element.p.text[1:].replace('.', ''))
                elif element.h3.text == 'Área construida' or element.h3.text == 'Área privada':
                    text = float(element.p.text[:-2].replace(' ', ''))
                elif element.h3.text == 'Parqueaderos':
                    text = int(element.p.text)
                elif element.h3.text == 'Barrio común':
                    text = element.p.text.upper()
                else:
                    text = element.p.text
                new_row[element.h3.text] = text
        except:
            obs_text = obs_text + ', '+  'sin info_4'
            new_row['Observaciones'] = obs_text

        try:
            card_block = soup.find_all('div', attrs={"class": "card-block"})
            try:
                lista_interiores_raw = card_block[3].find_all('ul', attrs={"class": "Ul-sctud2-0 kdIYsW inline-list-grid"})
                lista_interiores = list()
                for item in lista_interiores_raw:
                    lista_interiores.append(item.text)
                new_row['Interiores'] = ', '.join(lista_interiores)
            except:
                obs_text = obs_text + ', '+  'sin info_5.1'
                new_row['Observaciones'] = obs_text

            try:
                lista_exteriores_raw = card_block[4].find_all('ul', attrs={"class": "Ul-sctud2-0 kdIYsW inline-list-grid"})
                lista_exteriores = list()
                for item in lista_exteriores_raw:
                    lista_exteriores.append(item.text)
                new_row['Exteriores'] = ', '.join(lista_exteriores)
            except:
                obs_text = obs_text + ', '+  'sin info_5.2'
                new_row['Observaciones'] = obs_text
            try:
                lista_zonascomunes_raw = card_block[5].find_all('ul', attrs={"class": "Ul-sctud2-0 kdIYsW inline-list-grid"})
                lista_zonascomunes = list()
                for item in lista_zonascomunes_raw:
                    lista_zonascomunes.append(item.text)
                new_row['Zonas comunes'] = ', '.join(lista_zonascomunes)
            except:
                obs_text = obs_text + ', '+  'sin info_5.3'
                new_row['Observaciones'] = obs_text
        except:
            obs_text = obs_text + ', '+  'sin info_5'
            new_row['Observaciones'] = obs_text

        try:
            box_2 = soup.find('script', attrs={"id": "__NEXT_DATA__"})
            try:
                # Ubbicación / posición de coordenadas
                new_row['Ubicacion'] = json.loads(box_2.get_text())['props']['initialState']['realestate']['basic']['coordinates']
            except:
                new_row['Ubicacion'] = {'lon': None, 'lat': None}
                obs_text = obs_text + ', '+  'ubicación no disponible'
                new_row['Observaciones'] = obs_text
            try:
                # Imagenes
                list_images = json.loads(box_2.get_text())['props']['initialState']['realestate']['basic']['images']
                new_list_img = list()
                for img in list_images:
                    new_list_img.append(img['image'])
                new_list_img
                new_row['lista_imagenes'] = new_list_img
            except:
                new_row['lista_imagenes'] = 'NoInfo'
                obs_text = obs_text + ', '+  'imágenes no disponible'
                new_row['Observaciones'] = obs_text
            lat = new_row['Ubicacion']['lat']
            lon = new_row['Ubicacion']['lon']
            if lat is not None:
                new_row['G_maps'] = f'https://www.google.com/maps/place/{lat},{lon}'
            else:
                new_row['G_maps'] = 'NoInfo'
        except:
            pass

    except:
        obs_text = 'Sin acceso a enlace'
    new_row['Observaciones'] = obs_text
    return new_row

def reposo(time2sleep, n):
    clear_output(wait=True)
    print(f'>> REPOSO DE {time2sleep} SEGUNDOS, APARTAMENTOS BUSCADOS: {n}')
    print('------------------------------------------')
    time.sleep(time2sleep)
    return None

columnas = ['Código inmueble', 'Tipo', 'Habitaciones', 'Baños', 'Estrato', 'Precio', 'Antigüedad',
            'Área construida', 'Área privada', 'Valor administración', 'Parqueaderos', 'Sector',
            'Barrio común', 'Ubicacion','G_maps', 'Interiores', 'Exteriores', 'Zonas comunes', 'Valor arriendo',
            'Enlace', 'lista_imagenes', 'Observaciones']

def enlaces_to_df(lista_enlaces, columnas=columnas, verbose=False, new=False, no_reposo=50, seg_reposo=30):
    """
    dataframe
    lista enlaces
    bath_size
    """
    if len(lista_enlaces) < no_reposo:
        no_reposo = len(lista_enlaces)
        seg_reposo = 1
    lista_dict_aptos = list()
    if not new: df_aptos = pd.read_excel('main_aptos.xlsx')
    else: df_aptos = pd.DataFrame(columns=columnas)
    n = 0
    for enlace in lista_enlaces:
        cod = enlace.split('/')[-1]
        if cod not in list(df_aptos['Código inmueble']):
            print(f'{n+1}: {cod}')
            new_row = enlace_to_rowdf(enlace) # diccionario
            if verbose: print(new_row)
            print('--------')
            lista_dict_aptos.append(new_row)
            time.sleep(0.5)
            # cod = new_row['Código inmueble']
            n += 1
            if n % no_reposo == 0:
                if verbose: print(f'{n} apartamentos agregados')
                df_dict_aptos=pd.DataFrame(lista_dict_aptos)
                df_aptos = pd.concat([df_aptos, df_dict_aptos])
                df_aptos.drop_duplicates(subset=['Código inmueble'], keep='last', inplace=True)
                if verbose:
                    df_aptos.to_excel(dt.datetime.now().strftime('%Y%m%d_%H%M')+'_aptos'+'.xlsx', index=False) # checkpoint excel
                df_aptos.to_excel('main_aptos.xlsx', index=False) # sobreescribe principal
                reposo(seg_reposo, n) # descanso

        else:
            print(f'{cod} ya existe en base')

    print(f'FINALIZADO: {n} apartamentos agregados')
    df_aptos.to_excel(dt.datetime.now().strftime('%Y%m%d_%H%M')+'_aptos'+'.xlsx', index=False)
    # df_aptos.to_excel('main_aptos.xlsx', index=False)
    return df_aptos

In [99]:
browser = webdriver.Edge(EdgeChromiumDriverManager().install())
url = 'https://www.metrocuadrado.com/?search=form'
browser.get(url)
browser.maximize_window()
sleep(2)

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_52220\3953326603.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Edge(EdgeChromiumDriverManager().install())


In [100]:
filtro_tipo_inmueble = browser.find_elements(By.CLASS_NAME, "m2-select__control")[0]
actions_tipo_inmueble = ActionChains(browser)
actions_tipo_inmueble.send_keys_to_element(filtro_tipo_inmueble, Keys.ARROW_RIGHT)
actions_tipo_inmueble.send_keys(Keys.ARROW_DOWN)
actions_tipo_inmueble.send_keys(Keys.SPACE)
actions_tipo_inmueble.send_keys(Keys.ARROW_DOWN)
actions_tipo_inmueble.send_keys(Keys.SPACE)
actions_tipo_inmueble.perform()
sleep(2)

filtro_estado = browser.find_elements(By.CLASS_NAME, "m2-select__control")[1]
actions_filtro_estado = ActionChains(browser)
actions_filtro_estado.send_keys_to_element(filtro_estado, Keys.ARROW_RIGHT)
actions_filtro_estado.send_keys(Keys.ARROW_DOWN)
actions_filtro_estado.send_keys(Keys.ARROW_DOWN)
actions_filtro_estado.send_keys(Keys.TAB)
actions_filtro_estado.perform()
sleep(2)

selecting_city = browser.find_element(By.NAME, "location")
selecting_city.send_keys("Bogo")
sleep(1)

actions_selecting_city = ActionChains(browser)
actions_selecting_city.send_keys_to_element(selecting_city, Keys.ARROW_RIGHT)
actions_selecting_city.send_keys(Keys.ARROW_DOWN)
actions_selecting_city.send_keys(Keys.ARROW_DOWN)
actions_selecting_city.send_keys(Keys.ENTER)
actions_selecting_city.perform()
sleep(6)

ordenar = browser.find_elements(By.CLASS_NAME, "m2-select__control")[2]
actions = ActionChains(browser)
actions.send_keys_to_element(ordenar, Keys.ARROW_RIGHT)
actions.send_keys(Keys.ARROW_DOWN)
actions.send_keys(Keys.ARROW_DOWN)
actions.send_keys(Keys.ENTER)
actions.perform()

In [ ]:
# TODO: Terminar el proceso, por ahora sigue indefinidamente cargando la última página
run = True
pag = 1
while run:
    html = browser.page_source
    date_name = dt.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
    name_file = f'temporal_inmuebles_{date_name}.txt'
    with open(f'data_raw/{name_file}', 'w', encoding="utf-8") as file:
        file.write(html)
        print(f'Página {pag} exportada y guardada')
    pag += 1
    sleep(1)
    try:
        browser.find_element(By.CLASS_NAME, "item-icon-next").click()
        sleep(4)
    except:
        print(f'Proceso terminado, páginas agregadas: {pag-1}')
        run = False

In [4]:
path = "./data_raw"
dir_list = os.listdir(path)

lista_enlaces = list()
for doc in dir_list:
    out = txt_to_links(f'{path}/{doc}')
    lista_enlaces.append(out)
lista_enlaces = [item for sublist in lista_enlaces for item in sublist]
print(f'Total enlaces: {len(lista_enlaces)}')

./data_raw/temporal_inmuebles_2022_09_23_15_13_22.txt. Enlaces importados: 50
./data_raw/temporal_inmuebles_2022_09_23_15_14_08.txt. Enlaces importados: 50
./data_raw/temporal_inmuebles_2022_09_23_15_14_14.txt. Enlaces importados: 50
./data_raw/temporal_inmuebles_2022_09_23_15_14_19.txt. Enlaces importados: 50
./data_raw/temporal_inmuebles_2022_09_23_15_14_25.txt. Enlaces importados: 50
./data_raw/temporal_inmuebles_2022_09_23_15_14_30.txt. Enlaces importados: 50
./data_raw/temporal_inmuebles_2022_09_23_15_14_36.txt. Enlaces importados: 50
./data_raw/temporal_inmuebles_2022_09_23_15_14_41.txt. Enlaces importados: 50
./data_raw/temporal_inmuebles_2022_09_23_15_14_47.txt. Enlaces importados: 50
./data_raw/temporal_inmuebles_2022_09_23_15_14_52.txt. Enlaces importados: 50
./data_raw/temporal_inmuebles_2022_09_23_15_14_58.txt. Enlaces importados: 50
./data_raw/temporal_inmuebles_2022_09_23_15_15_04.txt. Enlaces importados: 50
./data_raw/temporal_inmuebles_2022_09_23_15_15_09.txt. Enlaces i

In [8]:
lista_enlaces = list(dict.fromkeys(lista_enlaces))
print(f'Eliminando enlaces duplicados...\n'
      f'Total enlaces: {len(lista_enlaces)}')

Eliminando enlaces duplicados...
Total enlaces: 9986


In [27]:
enlaces_to_df(lista_enlaces, new=False, verbose=False)

1: 10159-M3153758
--------
2: 10159-M3783750
--------
3: 3390-M3783729
--------
4: 2361-M3783845
--------
5: 4933-M3345528
--------
6: 4933-M3485899
--------
7: 3858-M3783752
--------
8: 10159-M3783789
--------
9: 707-M3783737
--------
10: 12503-M3783745
--------
11: 4933-M3618447
--------
12: 11813-M3783857
--------
13: 836-M3783731
--------
14: 4990-115543
--------
15: 4933-M3511707
--------
16: 2412-M3783837
--------
17: 3147-M3783778
--------
18: 4933-M3447417
--------
19: 34-M3783770
--------
20: 3147-M3783706
--------
21: 22-M3783764
--------
22: 507-M3783822
--------
23: 9816-M3606751
--------
24: 4933-M3348292
--------
25: 4933-M3511659
--------
26: 12041-M3342361
--------
27: 350-M3783863
--------
28: 4933-M3509698
--------
29: 34-57323
--------
30: 10093-M3783851
--------
31: 3390-M3783818
--------
32: 4933-M3618354
--------
33: 368-M3783751
--------
34: 3147-M3783741
--------
35: MC3783797
--------
36: 3008-M3783787
--------
37: 3340-M3783801
--------
38: 771-M3783774
------

,Código inmueble,Tipo,Habitaciones,Baños,Estrato,Precio,Antigüedad,Área construida,Área privada,Valor administración,...,Barrio común,Ubicacion,G_maps,Interiores,Exteriores,Zonas comunes,Valor arriendo,Enlace,lista_imagenes,Observaciones
0,10159-M3153758,Apartamento,2.0,2.0,6.0,630000000.0,Entre 5 y 10 años,65.0,65.0,550000.0,...,CHICO NORTE,"{'lon': -74.053154, 'lat': 4.684705}","https://www.google.com/maps/place/4.684705,-74...","Baño auxiliar, Calefacción, Chimenea, Citófono...","Ascensor, Baños de servicio, Circuito cerrado ...",Gimnasio,NaN,https://www.metrocuadrado.com/inmueble/venta-a...,[https://metrocuadrado.blob.core.windows.net/i...,
1,10159-M3783750,Apartamento,2.0,2.0,6.0,485000000.0,Más de 20 años,88.0,88.0,436000.0,...,CHICO,"{'lon': -74.05739, 'lat': 4.681892}","https://www.google.com/maps/place/4.681892,-74...","Baño auxiliar, Citófonos, Cocina tipo american...","Baños de servicio, Circuito cerrado de TV, Par...","Salón comunal, Zona para niños, Zonas verdes",NaN,https://www.metrocuadrado.com/inmueble/venta-a...,[https://metrocuadrado.blob.core.windows.net/i...,
2,3390-M3783729,Apartamento,3.0,5.0,6.0,980000000.0,Entre 10 y 20 años,155.0,155.0,900000.0,...,SANTA BARBARA,"{'lon': -74.04373, 'lat': 4.698853}","https://www.google.com/maps/place/4.698853,-74...","Apto para niños, Cuarto de servicio, Estudio o...","Area Terraza/Balcón 2 m², Acceso para discapac...",Zona de BBQ,NaN,https://www.metrocuadrado.com/inmueble/venta-a...,[https://metrocuadrado.blob.core.windows.net/i...,
3,2361-M3783845,Apartamento,2.0,3.0,4.0,490000000.0,Entre 0 y 5 años,81.0,81.0,NaN,...,COUNTRY CLUB USAQUEN,"{'lon': -74.048, 'lat': 4.72}","https://www.google.com/maps/place/4.72,-74.048","Calefacción, Hall de alcobas, Instalación de g...","Ascensor, Baños de servicio, Número de Ascenso...","Salón comunal, Zona de BBQ, Zonas verdes",NaN,https://www.metrocuadrado.com/inmueble/venta-a...,[https://metrocuadrado.blob.core.windows.net/i...,
4,4933-M3345528,Apartamento,3.0,3.0,4.0,795000000.0,Entre 0 y 5 años,121.0,106.0,430000.0,...,MAZUREN,"{'lon': -74.05995, 'lat': 4.7387724}","https://www.google.com/maps/place/4.7387724,-7...","Baño auxiliar, Calefacción, Chimenea, Citófono...","Ascensor, Baños de servicio, Parqueadero visit...",Gimnasio,NaN,https://www.metrocuadrado.com/inmueble/venta-a...,[https://metrocuadrado.blob.core.windows.net/i...,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9945,2469-M3767679,Apartamento,1.0,2.0,5.0,380000000.0,Entre 0 y 5 años,45.0,45.0,380000000.0,...,PASADENA,"{'lon': -74.05989, 'lat': 4.6953}","https://www.google.com/maps/place/4.6953,-74.0...","Apto para niños, Citófonos, Tipo de cortinas n...","Area Terraza/Balcón 2 m², Acceso para discapac...","Salón comunal, Zona de BBQ, Zona para niños",NaN,https://www.metrocuadrado.com/inmueble/venta-a...,[https://metrocuadrado.blob.core.windows.net/i...,
9946,3858-M3767678,Apartamento,1.0,1.0,6.0,700000000.0,Entre 0 y 5 años,57.0,57.0,300000.0,...,REFUGIO,"{'lon': -74.044174, 'lat': 4.668876}","https://www.google.com/maps/place/4.668876,-74...","Citófonos, Cocina integral, Estudio o bibliote...","Ascensor, Circuito cerrado de TV, Parqueadero ...",Zona de BBQ,NaN,https://www.metrocuadrado.com/inmueble/venta-a...,[https://metrocuadrado.blob.core.windows.net/i...,
9947,2832-M3767848,Apartamento,3.0,2.0,3.0,173400000.0,Entre 10 y 20 años,53.0,52.0,78000.0,...,VERBENAL SAN ANTONIO,"{'lon': -74.034904, 'lat': 4.758985}","https://www.google.com/maps/place/4.758985,-74...","Citófonos, Zona de lavandería, Instalación de ...","Circuito cerrado de TV, Conjunto cerrado, Núme...","Salón comunal, Zonas verdes",NaN,https://www.metrocuadrado.com/inmueble/venta-a...,[https://metrocuadrado.blob.core.windows.net/i...,
9948,MC3767549,Apartamento,3.0,2.0,4.0,380000000.0,Entre 10 y 20 años,78.0,72.0,255000.0,...,MAZUREN COLINA COLINA Y ALREDEDORES,"{'lon': -74.05909, 'lat': 4.7374043}","https://www.google.com/maps/place/4.7374043,-7...","Caracterís